In [1]:
import os
import pandas as pd
from datetime import datetime

In [2]:
def strToFloat(pdf, num_cols):
    for col in num_cols:
        pdf.loc[:, col] = pdf[col].astype(str).str.replace(',', '').astype(float)
    return pdf

def getTick(price):
    if price < 10:
        return 0.01
    elif price < 50:
        return 0.05
    elif price < 100:
        return 0.1
    elif price < 500:
        return 0.5
    elif price < 1000:
        return 1
    elif price < 3000:
        return 5

In [3]:
root_path = '/Users/fang/StockAnalysis/tmp/data'
today = datetime.now().strftime('%Y%m%d')
today = '20210611' # for testing

parent_path = root_path + os.sep + 'bsReport' + os.sep + today
smallDfList = []
largeDfList = []
for file in os.listdir(parent_path + os.sep + 'tpex') + os.listdir(parent_path + os.sep + 'twse'):
    ohlcDf = pd.read_csv(root_path + os.sep + 'ohlc' + os.sep + file)
    dateDash = '2021-06-11'
    
    ohlcDf = ohlcDf[ohlcDf['日期']==dateDash]
    if not len(ohlcDf):
        continue
    ohlcRecord = ohlcDf.iloc[0]
    
    if ohlcRecord['漲跌幅'] > 9.45: # 假設大漲
        stockDf = None
        if os.path.exists(parent_path + os.sep + 'tpex' + os.sep + file):
            stockDf = pd.read_csv(parent_path + os.sep + 'tpex' + os.sep + file)
        elif os.path.exists(parent_path + os.sep + 'twse' + os.sep + file):
            stockDf = pd.read_csv(parent_path + os.sep + 'twse' + os.sep + file)
        elif not stockDf:
            continue
        stockDf = strToFloat(stockDf.dropna(), ['買進股數', '賣出股數'])
        
        buyDf = stockDf[stockDf['買進股數'] > 0].copy()
        buyDf.loc[:, '買進金額'] = buyDf['價格'] * buyDf['買進股數']
        buyDf = buyDf.loc[:, ['券商', '買進金額', '買進股數']].groupby("券商").sum().reset_index()
        buyDf.loc[:, '買進均價'] = buyDf['買進金額'] / buyDf['買進股數']
        buyDf = buyDf.dropna()

        sellDf = stockDf[stockDf['賣出股數'] > 0].copy()
        sellDf.loc[:, '賣出金額'] = sellDf['價格'] * sellDf['賣出股數']
        sellDf = sellDf.loc[:, ['券商', '賣出金額', '賣出股數']].groupby("券商").sum().reset_index()
        sellDf.loc[:, '賣出均價'] = sellDf['賣出金額'] / sellDf['賣出股數']
        sellDf = sellDf.dropna()
        
        mergeDf = pd.merge(buyDf, sellDf, on='券商', how='outer').fillna(0.0)
        totalVolume = mergeDf['買進股數'].sum()
        mergeDf['買超股數'] = mergeDf['買進股數'] - mergeDf['賣出股數']
        mergeDf['買超張數'] = mergeDf['買進股數'] / 1000
        mergeDf['買超比率'] = mergeDf['買超股數'] / totalVolume * 100
        mergeDf.loc[:, '股票代號'] = file.replace('.csv', '')
        mergeDf.loc[:, '成交量'] = ohlcRecord['成交股數']/1000
        mergeDf.loc[:, '成交金額(萬)'] = ohlcRecord['成交金額']/10000
        mergeDf.loc[:, '開盤價'] = ohlcRecord['開盤價']
        mergeDf.loc[:, '最高價'] = ohlcRecord['最高價']
        mergeDf.loc[:, '最低價'] = ohlcRecord['最低價']
        mergeDf.loc[:, '收盤價'] = ohlcRecord['收盤價']
        mergeDf.loc[:, '漲跌幅'] = ohlcRecord['漲跌幅']
        
        highPrice = ohlcRecord['最高價'] - getTick(ohlcRecord['最高價'])
        
        # 累積小量買超券商(>5%), 且買均價大於
        smallDf = mergeDf[(mergeDf['買超比率']>1.5) & (mergeDf['買進均價']>=highPrice)]
        if len(smallDf) and smallDf['買超比率'].sum() > 5:
            smallDfList.append(smallDf)
            
        # 單一大量買超券商(>5%), 且買均價大於
        
        largeDf = mergeDf[(mergeDf['買超比率']>5) & (mergeDf['買進均價']>=highPrice)]
        if len(largeDf):
            largeDfList.append(largeDf)
        
    else: # 假設沒大漲
        pass
    
stockInfoDf = pd.read_csv(root_path + os.sep + 'common' + os.sep + 'stockList.csv')
stockInfoDf = stockInfoDf[['股票代號', '股票名稱']]
stockInfoDf.loc[:, '股票代號'] = stockInfoDf['股票代號'].apply(lambda x: str(x))

brokerDf = pd.read_csv(root_path + os.sep + 'common' + os.sep + 'brokerList.csv')
brokerDf.columns = ['券商', '券商名稱', '開業日', '地址', '電話']
brokerDf = brokerDf[['券商', '券商名稱']]

resultDf = pd.concat(largeDfList+smallDfList, axis=0, ignore_index=True, sort=False).drop_duplicates()
resultDf = pd.merge(resultDf, stockInfoDf, on='股票代號')
resultDf = pd.merge(resultDf, brokerDf, on='券商')
resultDf = resultDf[['股票代號', '股票名稱', '券商名稱', '成交量', '成交金額(萬)', '買超張數', '買超比率', '買進均價', '收盤價', '開盤價', '漲跌幅', '最高價', '最低價']]

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(resultDf.sort_values(['成交金額(萬)', '股票代號', '買超比率']))

# For single stock

In [59]:
stock = '5608'
stockType = 'twse'

ohlcDf = pd.read_csv(root_path + os.sep + 'ohlc' + os.sep + stock + '.csv')
dateDash = '2021-06-11'

ohlcDf = ohlcDf[ohlcDf['日期']==dateDash]
ohlcRecord = ohlcDf.iloc[0]

stockDf = pd.read_csv(parent_path + os.sep + stockType + os.sep + stock + '.csv')
stockDf = strToFloat(stockDf.dropna(), ['買進股數', '賣出股數'])
        
buyDf = stockDf[stockDf['買進股數'] > 0].copy()
buyDf.loc[:, '買進金額'] = buyDf['價格'] * buyDf['買進股數']
buyDf = buyDf.loc[:, ['券商', '買進金額', '買進股數']].groupby("券商").sum().reset_index()
buyDf.loc[:, '買進均價'] = buyDf['買進金額'] / buyDf['買進股數']
buyDf = buyDf.dropna()

sellDf = stockDf[stockDf['賣出股數'] > 0].copy()
sellDf.loc[:, '賣出金額'] = sellDf['價格'] * sellDf['賣出股數']
sellDf = sellDf.loc[:, ['券商', '賣出金額', '賣出股數']].groupby("券商").sum().reset_index()
sellDf.loc[:, '賣出均價'] = sellDf['賣出金額'] / sellDf['賣出股數']
sellDf = sellDf.dropna()

mergeDf = pd.merge(buyDf, sellDf, on='券商', how='outer').fillna(0.0)
totalVolume = mergeDf['買進股數'].sum()
mergeDf['買超股數'] = mergeDf['買進股數'] - mergeDf['賣出股數']
mergeDf['買超張數'] = mergeDf['買進股數'] / 1000
mergeDf['買超比率'] = mergeDf['買超股數'] / totalVolume * 100
mergeDf.loc[:, '股票代號'] = stock
mergeDf.loc[:, '成交量'] = ohlcRecord['成交股數']/1000
mergeDf.loc[:, '成交金額(萬)'] = ohlcRecord['成交金額']/10000
mergeDf.loc[:, '開盤價'] = ohlcRecord['開盤價']
mergeDf.loc[:, '最高價'] = ohlcRecord['最高價']
mergeDf.loc[:, '最低價'] = ohlcRecord['最低價']
mergeDf.loc[:, '收盤價'] = ohlcRecord['收盤價']
mergeDf.loc[:, '漲跌幅'] = ohlcRecord['漲跌幅']
    
stockInfoDf = pd.read_csv(root_path + os.sep + 'common' + os.sep + 'stockList.csv')
stockInfoDf = stockInfoDf[['股票代號', '股票名稱']]
stockInfoDf.loc[:, '股票代號'] = stockInfoDf['股票代號'].apply(lambda x: str(x))

brokerDf = pd.read_csv(root_path + os.sep + 'common' + os.sep + 'brokerList.csv')
brokerDf.columns = ['券商', '券商名稱', '開業日', '地址', '電話']
brokerDf = brokerDf[['券商', '券商名稱']]

largeResultDf = mergeDf
largeResultDf = pd.merge(largeResultDf, stockInfoDf, on='股票代號')
largeResultDf = pd.merge(largeResultDf, brokerDf, on='券商')
largeResultDf = largeResultDf[['股票代號', '股票名稱', '券商名稱', '成交量', '成交金額(萬)', '買超張數', '買超比率', '買進均價', '收盤價', '開盤價', '漲跌幅', '最高價', '最低價']]
largeResultDf.sort_values('買超比率', ascending=False)['買超比率']

# Price limit calculation

In [326]:
# price limit calculation

priceRange = {(5, 10): (0.01, 0.05), 
              (10, 50): (0.05, 0.1),
              (50, 100): (0.1, 0.5),
              (100, 500): (0.5, 1),
              (500, 1000): (1, 5),
              (1000, 3000): (5, 5)}
ratioLimit = 0.1
minRatio = 10
count = 0
for price in priceRange:
    start_price = price[0]
    end_price = price[1]
    cur_tick = priceRange[price][0]
    next_tick = priceRange[price][1]
    old_price = start_price
    while old_price < end_price:
        new_price = old_price
        ratio = round((new_price - old_price) / old_price, 5)
        while ratio < ratioLimit:
            new_price = round(new_price+cur_tick, 2)
            ratio = round((new_price - old_price) / old_price, 5)
        if ratio > ratioLimit:
            new_price -= cur_tick
        if new_price > end_price:
            tmp_price = end_price
            while tmp_price <= new_price:
                tmp_price = round(tmp_price + next_tick, 5)
            new_price = tmp_price - next_tick
        ratio = round((new_price - old_price) / old_price, 5) 
        if round(ratio*100, 2) < minRatio:
            minRatio = round(ratio*100, 2)
        print(str(round(old_price, 2)) + '\t' + str(round(new_price, 2)) + '\t' + str(round(ratio*100, 2))+'%')
        old_price = round(old_price+cur_tick, 2)
        count += 1

5	5.5	10.0%
5.01	5.51	9.98%
5.02	5.52	9.96%
5.03	5.53	9.94%
5.04	5.54	9.92%
5.05	5.55	9.9%
5.06	5.56	9.88%
5.07	5.57	9.86%
5.08	5.58	9.84%
5.09	5.59	9.82%
5.1	5.61	10.0%
5.11	5.62	9.98%
5.12	5.63	9.96%
5.13	5.64	9.94%
5.14	5.65	9.92%
5.15	5.66	9.9%
5.16	5.67	9.88%
5.17	5.68	9.87%
5.18	5.69	9.85%
5.19	5.7	9.83%
5.2	5.72	10.0%
5.21	5.73	9.98%
5.22	5.74	9.96%
5.23	5.75	9.94%
5.24	5.76	9.92%
5.25	5.77	9.9%
5.26	5.78	9.89%
5.27	5.79	9.87%
5.28	5.8	9.85%
5.29	5.81	9.83%
5.3	5.83	10.0%
5.31	5.84	9.98%
5.32	5.85	9.96%
5.33	5.86	9.94%
5.34	5.87	9.93%
5.35	5.88	9.91%
5.36	5.89	9.89%
5.37	5.9	9.87%
5.38	5.91	9.85%
5.39	5.92	9.83%
5.4	5.94	10.0%
5.41	5.95	9.98%
5.42	5.96	9.96%
5.43	5.97	9.95%
5.44	5.98	9.93%
5.45	5.99	9.91%
5.46	6.0	9.89%
5.47	6.01	9.87%
5.48	6.02	9.85%
5.49	6.03	9.84%
5.5	6.05	10.0%
5.51	6.06	9.98%
5.52	6.07	9.96%
5.53	6.08	9.95%
5.54	6.09	9.93%
5.55	6.1	9.91%
5.56	6.11	9.89%
5.57	6.12	9.87%
5.58	6.13	9.86%
5.59	6.14	9.84%
5.6	6.16	10.0%
5.61	6.17	9.98%
5.62	6.18	9.96%
5.63	6.19	

40.25	44.25	9.94%
40.3	44.3	9.93%
40.35	44.35	9.91%
40.4	44.4	9.9%
40.45	44.45	9.89%
40.5	44.55	10.0%
40.55	44.6	9.99%
40.6	44.65	9.98%
40.65	44.7	9.96%
40.7	44.75	9.95%
40.75	44.8	9.94%
40.8	44.85	9.93%
40.85	44.9	9.91%
40.9	44.95	9.9%
40.95	45.0	9.89%
41.0	45.1	10.0%
41.05	45.15	9.99%
41.1	45.2	9.98%
41.15	45.25	9.96%
41.2	45.3	9.95%
41.25	45.35	9.94%
41.3	45.4	9.93%
41.35	45.45	9.92%
41.4	45.5	9.9%
41.45	45.55	9.89%
41.5	45.65	10.0%
41.55	45.7	9.99%
41.6	45.75	9.98%
41.65	45.8	9.96%
41.7	45.85	9.95%
41.75	45.9	9.94%
41.8	45.95	9.93%
41.85	46.0	9.92%
41.9	46.05	9.9%
41.95	46.1	9.89%
42.0	46.2	10.0%
42.05	46.25	9.99%
42.1	46.3	9.98%
42.15	46.35	9.96%
42.2	46.4	9.95%
42.25	46.45	9.94%
42.3	46.5	9.93%
42.35	46.55	9.92%
42.4	46.6	9.91%
42.45	46.65	9.89%
42.5	46.75	10.0%
42.55	46.8	9.99%
42.6	46.85	9.98%
42.65	46.9	9.96%
42.7	46.95	9.95%
42.75	47.0	9.94%
42.8	47.05	9.93%
42.85	47.1	9.92%
42.9	47.15	9.91%
42.95	47.2	9.89%
43.0	47.3	10.0%
43.05	47.35	9.99%
43.1	47.4	9.98%
43.15	47.45	9.96%


130.0	143.0	10.0%
130.5	143.5	9.96%
131.0	144.0	9.92%
131.5	144.5	9.89%
132.0	145.0	9.85%
132.5	145.5	9.81%
133.0	146.0	9.77%
133.5	146.5	9.74%
134.0	147.0	9.7%
134.5	147.5	9.66%
135.0	148.5	10.0%
135.5	149.0	9.96%
136.0	149.5	9.93%
136.5	150.0	9.89%
137.0	150.5	9.85%
137.5	151.0	9.82%
138.0	151.5	9.78%
138.5	152.0	9.75%
139.0	152.5	9.71%
139.5	153.0	9.68%
140.0	154.0	10.0%
140.5	154.5	9.96%
141.0	155.0	9.93%
141.5	155.5	9.89%
142.0	156.0	9.86%
142.5	156.5	9.83%
143.0	157.0	9.79%
143.5	157.5	9.76%
144.0	158.0	9.72%
144.5	158.5	9.69%
145.0	159.5	10.0%
145.5	160.0	9.97%
146.0	160.5	9.93%
146.5	161.0	9.9%
147.0	161.5	9.86%
147.5	162.0	9.83%
148.0	162.5	9.8%
148.5	163.0	9.76%
149.0	163.5	9.73%
149.5	164.0	9.7%
150.0	165.0	10.0%
150.5	165.5	9.97%
151.0	166.0	9.93%
151.5	166.5	9.9%
152.0	167.0	9.87%
152.5	167.5	9.84%
153.0	168.0	9.8%
153.5	168.5	9.77%
154.0	169.0	9.74%
154.5	169.5	9.71%
155.0	170.5	10.0%
155.5	171.0	9.97%
156.0	171.5	9.94%
156.5	172.0	9.9%
157.0	172.5	9.87%
157.5	173.0	9.84%

760	836	10.0%
761	837	9.99%
762	838	9.97%
763	839	9.96%
764	840	9.95%
765	841	9.93%
766	842	9.92%
767	843	9.91%
768	844	9.9%
769	845	9.88%
770	847	10.0%
771	848	9.99%
772	849	9.97%
773	850	9.96%
774	851	9.95%
775	852	9.93%
776	853	9.92%
777	854	9.91%
778	855	9.9%
779	856	9.88%
780	858	10.0%
781	859	9.99%
782	860	9.97%
783	861	9.96%
784	862	9.95%
785	863	9.94%
786	864	9.92%
787	865	9.91%
788	866	9.9%
789	867	9.89%
790	869	10.0%
791	870	9.99%
792	871	9.98%
793	872	9.96%
794	873	9.95%
795	874	9.94%
796	875	9.93%
797	876	9.91%
798	877	9.9%
799	878	9.89%
800	880	10.0%
801	881	9.99%
802	882	9.98%
803	883	9.96%
804	884	9.95%
805	885	9.94%
806	886	9.93%
807	887	9.91%
808	888	9.9%
809	889	9.89%
810	891	10.0%
811	892	9.99%
812	893	9.98%
813	894	9.96%
814	895	9.95%
815	896	9.94%
816	897	9.93%
817	898	9.91%
818	899	9.9%
819	900	9.89%
820	902	10.0%
821	903	9.99%
822	904	9.98%
823	905	9.96%
824	906	9.95%
825	907	9.94%
826	908	9.93%
827	909	9.92%
828	910	9.9%
829	911	9.89%
830	913	10.0%
831	914	9.99%

In [74]:
os.listdir('abc')

FileNotFoundError: [Errno 2] No such file or directory: 'abc'

In [ ]:
os.path.exists